In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os, sys
from pathlib import Path
import seaborn as sns
import numpy as np
import glob
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score, accuracy_score, auc, precision_recall_fscore_support, pairwise, f1_score, log_loss, make_scorer
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm

#importin xg boost and all needed otherstuff
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier #conda install -c conda-forge xgboost to install
##adding these, lets see if it helps with xgboost crash
os.environ['KMP_DUPLICATE_LIB_OK']='True'

#reducing warnings that are super common in my model
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.simplefilter(action='ignore') #ignore all warnings

RANDOM_STATE = 15485867

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime

In [2]:
#patients of interest from rotation_cohort_generation
from parameters import final_pt_df_v, date, repository_path, lower_window, upper_window, folder, date, time_col, time_var, patient_df

#patients of interest from rotation_cohort_generation
final_pt_df2 = final_pt_df_v #pd.read_csv('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_final_pt_df2.csv'%(most_updated_patient_df), index_col=0)
del(final_pt_df_v)

patients= list(final_pt_df2['subject_id'].unique())
hadm_id= list(final_pt_df2['hadm_id'].unique())
icustay_id= list(final_pt_df2['icustay_id'].unique())
icustay_id= [int(x) for x in icustay_id]

save_boolean=False

time: 1.12 s


## importing x and y train and test

In [3]:

def data_import(allFiles):
    """
    function to import x_train, x_test, y_train, and y_test using glob of the data/final folder.
    """
    for name in allFiles:
        if 'test' in name:
            if 'x_' in name:
                x_test = pd.read_csv(name,  index_col=0)
            else:
                 y_test = pd.read_csv(name,  index_col=0)
        elif 'train' in name:
            if 'x_' in name:
                x_train = pd.read_csv(name,  index_col=0)
            else:
                 y_train = pd.read_csv(name,  index_col=0)
    return(x_train, x_test, y_train, y_test)


time: 7.64 ms


In [4]:
#importing x and y train and test

allFiles_24 = glob.glob(str(repository_path)+ '/data/final/%s/'%('24_hr_window') + "*.csv")
allFiles_48 = glob.glob(str(repository_path)+ '/data/final/%s/'%('48_hr_window') + "*.csv")
allFiles_72 = glob.glob(str(repository_path)+ '/data/final/%s/'%('72_hr_window') + "*.csv")

x_train_24, x_test_24, y_train_24, y_test_24= data_import(allFiles_24)
x_train_48, x_test_48, y_train_48, y_test_48= data_import(allFiles_48)
x_train_72, x_test_72, y_train_72, y_test_72= data_import(allFiles_72)

time: 416 ms


## importing all models and storing them in dictionary

In [5]:
def load_model(filename, timewindow):
    import pickle
    loaded_modle= pickle.load(open(filename, 'rb'))
    return(loaded_modle)

time: 1.63 ms


In [6]:
models_24 = glob.glob(str(repository_path)+ '/models/%s/'%('24_hr_window')+'*')
models_48 = glob.glob(str(repository_path)+ '/models/%s/'%('48_hr_window')+'*')
models_72 = glob.glob(str(repository_path)+ '/models/%s/'%('72_hr_window')+'*')

models_24_dic={}
models_48_dic={}
models_72_dic={}

for model in models_24:
    models_24_dic.update( {model.strip('.sav').split('_')[-1] : load_model(model, '24_hr_window')} )
    
for model in models_48:
    models_48_dic.update( {model.strip('.sav').split('_')[-1] : load_model(model, '48_hr_window')} )

for model in models_72:
    models_72_dic.update( {model.strip('.sav').split('_')[-1] : load_model(model, '72_hr_window')} )

time: 550 ms


## permutation stuff

* We can control for dependent variables by stratifying the output and only permuting within each stratum. (Yeh 2000, Noreen 1989)
* In this case, we’ll stratify each $o^i_A$,$o^j_B$
* Let $o_A = {o^1_A, ... , o^n_A}$ and $o_B = {o^1_B, ... , o^n_B}$ be the output of the two systems on the same input.
* Start with $X= o_A and Y=o_B$
* Repeat R times: randomly flip each $o^i_A, o^j_B$  between X and Y with probability $\frac{1}{2}$. Calculate $t(X,Y)$.
* Let r be the number of times that $t(X,Y)≥t(o_A,o_B)$
* As R → $∞$, $\frac{r+1}{R+1}$ approaches the significance level.

--taken from presentation: Statistical Hypothesis Tests for NLP or: Approximate Randomization for Fun and Profit by William Morgan

* Null hypothesis: there is no difference between our model using N=72hr of data vs our model using N=24hr of data

* We’ll run two systems on the same input and see how their output differs.

* In the binary comparison case, with threshold 0.05, validity tells us that we’ll falsely reject the null hypothesis (make a type I error) 5% of the time.

In [312]:
def perm_test_fxn(out1,out2, absolute=False, verbose=True, N=1000):
    row = len(out1)

    mean1 = np.mean(out1);
    mean2 = np.mean(out2)

    pmean1 = [0] *N; pmean2 = [0] *N #initializing array of size N that will be replaced in loop
    perm1 = out1;
    perm2 = out2;

    for i in range(N):
        coins = np.random.rand(row);
        ind1 = 0.5 <=coins;
        ind2 = 0.5 >coins;
        #assinging random indicies to be from out1 and out2. 
        
        #initialization
#         perm1 = out1
#         perm2 = out2
#         perm1 = np.random.rand(row)
#         perm2 = np.random.rand(row)
        perm1 = np.ones(row)
        perm2 = np.ones(row)
        
        #assiging values from the resampled pair
        perm1[ind1] =out1[ind1];
        perm1[ind2] =out2[ind2];
        perm2[ind1] =out2[ind1];
        perm2[ind2] =out1[ind2];
        
        pmean1[i] = np.mean(perm1)
        pmean2[i] = np.mean(perm2)
        
    if absolute==False:
        p = rsig((np.array(pmean1) - np.array(pmean2)), (np.array(mean1) - np.array(mean2)) )
    else:
        p = rsig(abs(np.array(pmean1) - np.array(pmean2)), abs(np.array(mean1) - np.array(mean2)) ); # one side test

    if verbose==True:
        print('rand paired sig test on mean p=%f\n' % (p))
    return (p);

time: 43.1 ms


In [308]:
sum(out2==1)

0

time: 10.3 ms


In [313]:
model_names=[]
mean_72=[]
mean_24=[]
delta_mean=[]
p_list=[]
for key in models_72_dic.keys():
    out1=models_72_dic[key].predict_proba(np.array(x_test_72))[:,1]
    out2=models_24_dic[key].predict_proba(np.array(x_test_24))[:,1]
    print("%s 72,24hr, dif: %.3f, %.3f, %.3f"% (key, np.mean(out1),np.mean(out2), np.mean(out1)-np.mean(out2)))
    model_names.append(key)
    mean_72.append(np.mean(out1))
    mean_24.append(np.mean(out2))
    delta_mean.append(np.mean(out1)-np.mean(out2))
    p=perm_test_fxn(out1,out2, absolute=True, verbose=True, N=5000)
    p_list.append(p)

delta_dic={'model':model_names,
           'mean_72':mean_72,
           'mean_24':mean_24,
           'mean_delta':delta_mean,
           'resample_p':p_list}

rf 72,24hr, dif: 0.236, 0.238, -0.002
rand paired sig test on mean p=0.322336

knn 72,24hr, dif: 0.186, 0.185, 0.001
rand paired sig test on mean p=0.643871

logreg 72,24hr, dif: 0.232, 0.235, -0.003
rand paired sig test on mean p=0.036393

xgboost 72,24hr, dif: 0.231, 0.236, -0.005
rand paired sig test on mean p=0.004399

ensemble 72,24hr, dif: 0.233, 0.236, -0.004
rand paired sig test on mean p=0.009998

svc 72,24hr, dif: 0.230, 0.236, -0.006
rand paired sig test on mean p=0.000400

time: 11.7 s


In [315]:
pd.DataFrame(delta_dic).round(3) #run with initializing to rand or 1's

,model,mean_72,mean_24,mean_delta,resample_p
0,rf,0.236,0.238,-0.002,0.322
1,knn,0.186,0.185,0.001,0.644
2,logreg,0.232,0.235,-0.003,0.036
3,xgboost,0.231,0.236,-0.005,0.004
4,ensemble,0.233,0.236,-0.004,0.010
5,svc,0.230,0.236,-0.006,0.000


time: 11.2 ms


In [302]:
pd.DataFrame(delta_dic).round(3) #run with initializing to rand or 1's

,model,mean_72,mean_24,mean_delta,resample_p
0,rf,0.236,0.238,-0.002,0.329
1,knn,0.186,0.185,0.001,0.654
2,logreg,0.232,0.235,-0.003,0.033
3,xgboost,0.231,0.236,-0.005,0.002
4,ensemble,0.233,0.236,-0.004,0.008
5,svc,0.230,0.236,-0.006,0.000


time: 12.7 ms


* using abs or not:
 * here we are comparing the difference in means between (meanP(model_72) - meanP(model_24))=mean_model_diff and each resample of (meanP(perm1) - meanP(perm2))=perm_diff[i]
 * we count the # of times the perm_diff >= man_model_diff = count(permdiff)
 * we then (count(perm_diff)+1)/ #resamples+1 = p.
 * so if we use absolute value of the differences


In [272]:
pd.DataFrame(delta_dic)

,model,mean_72,mean_24,mean_delta
0,rf,0.236315,0.237831,-0.001516
1,knn,0.186090,0.185406,0.000684
2,logreg,0.231815,0.235129,-0.003314
3,xgboost,0.231334,0.236067,-0.004733
4,ensemble,0.232640,0.236308,-0.003668
5,svc,0.230368,0.236199,-0.005831


time: 10.5 ms


rf 72,24hr, dif: 0.236, 0.238, -0.002
knn 72,24hr, dif: 0.186, 0.185, 0.001
logreg 72,24hr, dif: 0.232, 0.235, -0.003
xgboost 72,24hr, dif: 0.231, 0.236, -0.005
ensemble 72,24hr, dif: 0.233, 0.236, -0.004
svc 72,24hr, dif: 0.230, 0.236, -0.006
time: 5.22 s


dict_keys(['/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-opensource/models/72_hr_window/finalized_svc.sav'])

time: 3.12 ms


In [261]:
list(models_72_dic.keys())

['rf', 'knn', 'logreg', 'xgboost', 'ensemble', 'svc']

time: 3.15 ms


In [233]:
def rsig(perm_diff, mean_model_diff):

    R = len(perm_diff)
    r = sum(perm_diff >= mean_model_diff)
#     print('R,r: ', R,r)
    p = (r+1.)/(R+1.)
#     print('p: ', p)
    ## p = r/R
    return (p);


time: 4.2 ms


In [202]:
def perm_test_fxn2(out1,out2):
    '''same as perm_test_fxn but more verbose for bugfixing'''

    N=5
    row = len(out1)

    mean1 = np.mean(out1);
    mean2 = np.mean(out2)

    pmean1 = [0] *N; pmean2 = [0] *N #initializing array of size N that will be replaced in loop
    perm1 = out1;
    perm2 = out2;

    for i in range(N):
        coins = np.random.rand(row);
    #     print('coins: ', coins)
        print('lencoins, out1',len(coins), print(len(perm1)))
        ## only pick one half 

        ind1 = 0.5 <=coins;
        ind2 = 0.5 >coins;
        print('coins: ', coins)
        print('ind1: ', ind1)
        print('ind2', ind2)
        #the lines above are a way of picking random indicies to sample from 
        
        perm1 = np.ones(row)
        perm2 = np.ones(row)

        perm1[ind1] =out1[ind1];
        perm1[ind2] =out2[ind2];

        perm2[ind1] =out2[ind1];
        perm2[ind2] =out1[ind2];

        pmean1[i] = np.mean(perm1)
        pmean2[i] = np.mean(perm2)
        print('perm1[ind1]:',perm1[ind1] )
        print('perm1: ', perm1)
        print('perm2[ind1]:',perm2[ind1] )
        print('perm2: ', perm2)
        #print('pmean1[i]:',pmean1[i])
        print('pmean1:',pmean1)
        #print('pmean2[i]:',pmean2[i])
        print('pmean2:',pmean2)
#         del(ind1,ind2)
    print('final_perm1: ', perm1)
    print('final_perm2: ', perm2)
 


time: 61.5 ms


In [241]:
#df_dic['rf'],x=x_test_24,y=y_test_24
out1=models_72_dic['rf'].predict_proba(np.array(x_test_72))[:,1]
out2=models_24_dic['rf'].predict_proba(np.array(x_test_24))[:,1]
perm_test_fxn(out1,out2)

rand paired sig test on mean p=0.848152



0.8481518481518482

time: 473 ms


In [229]:

np.mean(out1),
np.mean(out2)

0.23133396

time: 2.94 ms


In [228]:
np.mean(out2)

0.23606709

time: 2.7 ms


In [77]:
np.mean(out1-out2)

0.0

time: 2.9 ms


In [48]:
from statistics import mean
mean(out2[:5])

TypeError: can't convert type 'float32' to numerator/denominator

time: 12.9 ms


In [54]:
randPairedSigTest(out1,out2, N=1000)

1000
rand paired sig test on mean p=1.000000



1.0

time: 4.83 s


In [27]:
out1

array([0.24957651, 0.25126702, 0.11935172, ..., 0.11380261, 0.6149088 ,
       0.06434408], dtype=float32)

time: 3.01 ms
